# Предсказать форму облаков

## Подключение библиотек

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import requests

from imgaug.augmentables.segmaps import SegmentationMapOnImage
from PIL import Image
from io import BytesIO